In [1]:
import storms
from cxotime import CxoTime, TimeDelta
import astropy.units as u
import numpy as np
import jinja2
import re
from pathlib import Path
from docutils.core import publish_file

In [2]:
#shutdown = "2022-03-28T12:40:00"
#startup = "2022-04-04T05:59:00"
shutdown = "2017-09-07T02:31:00"
startup = "2017-09-09T06:04:00"

In [3]:
dt = TimeDelta(2.0*u.d)

In [4]:
shutdown = CxoTime(shutdown)
startup = CxoTime(startup)

In [5]:
storm_date = shutdown.yday[:8]

In [6]:
storm_dir = Path(f"storm_{storm_date.replace(':','_')}")
if not storm_dir.exists():
    storm_dir.mkdir()

In [7]:
sw = storms.SolarWind(shutdown-dt, startup+dt)

In [8]:
dpe = sw.plot_electrons()
dpe.add_vline(shutdown, color='k')
dpe.add_vline(startup, color='k')
dpe.add_text(shutdown+TimeDelta(0.1*u.day), 1.0, "SHUTDOWN", rotation="vertical")
dpe.add_text(startup+TimeDelta(0.1*u.day), 1.0, "STARTUP", rotation="vertical")
dpe.savefig(storm_dir / "electrons_vs_time.png")

In [9]:
dpp = sw.plot_protons()
dpp.add_vline(shutdown, color='k')
dpp.add_vline(startup, color='k')
dpp.add_text(shutdown+TimeDelta(0.1*u.day), 1.0, "SHUTDOWN", rotation="vertical")
dpp.add_text(startup+TimeDelta(0.1*u.day), 1.0, "STARTUP", rotation="vertical")
dpp.savefig(storm_dir / "protons_vs_time.png")

In [10]:
gpp = sw.plot_hrc()
gpp.add_vline(shutdown, color='k')
gpp.add_vline(startup, color='k')
gpp.add_text(shutdown+TimeDelta(0.1*u.day), 1.0, "SHUTDOWN", rotation="vertical")
gpp.add_text(startup+TimeDelta(0.1*u.day), 1.0, "STARTUP", rotation="vertical")
gpp.savefig(storm_dir / "hrc_vs_time.png")

In [11]:
times = [(shutdown-(2-i)*TimeDelta(12*u.hr)).secs for i in range(4)]
times.append(CxoTime("2017:249:08:00:00").secs)
times.append(CxoTime("2017:249:23:00:00").secs)
times.sort()
fig = sw.plot_proton_spectra(times)
fig.axes[1].axvline(shutdown.plot_date, color='k')
fig.axes[1].axvline(startup.plot_date, color='k')
fig.axes[1].text((shutdown+TimeDelta(0.1*u.day)).plot_date, 15.0, "SHUTDOWN", 
                 rotation="vertical", fontsize=18)
fig.axes[1].text((startup+TimeDelta(0.1*u.day)).plot_date, 15.0, "STARTUP", 
                 rotation="vertical", fontsize=18)
fig.axes[1].set_ylim(10.0, None)
fig.savefig(storm_dir / "proton_spectra.png")

In [12]:
fig = sw.scatter_plots()
idx = np.searchsorted(sw.times, shutdown.secs)
row = sw.table[idx]
fig.axes[0].plot(row["p3"], row["hrc_shield"], 'x', mew=3, ms=20, color='C3', label="Shutdown")
fig.axes[0].legend(fontsize=18)
fig.axes[1].plot(row["p3"], row["de1"], 'x', mew=3, ms=20, color='C3')
fig.savefig(storm_dir / "scatter_plots.png")

In [13]:
storm_template_file = 'storm_template.rst'

storm_template = open(Path("/Users/jzuhone/Source/storms/templates") / storm_template_file).read()
storm_template = re.sub(r' %}\n', ' %}', storm_template)

context = {"shutdown": "YES",
           "storm_date": storm_date,
           "load": "SEP0417A",
           "trigger": "HRC Anti-Co Shield",
           "shutdown_time": shutdown.yday,
           "startup_time": startup.yday}

outfile = storm_dir / "index.rst"

template = jinja2.Template(storm_template)

with open(outfile, "w") as f:
    f.write(template.render(**context))

In [14]:
infile = str(outfile)
outfile = str(storm_dir / 'index.html')
publish_file(source_path=infile, destination_path=outfile,
             writer_name="html")

'<?xml version="1.0" encoding="utf-8" ?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta name="generator" content="Docutils 0.17.1: http://docutils.sourceforge.net/" />\n<title>Storm: 2017:250</title>\n<style type="text/css">\n\n/*\n:Author: David Goodger (goodger@python.org)\n:Id: $Id: html4css1.css 7952 2016-07-26 18:15:59Z milde $\n:Copyright: This stylesheet has been placed in the public domain.\n\nDefault cascading style sheet for the HTML output of Docutils.\n\nSee http://docutils.sf.net/docs/howto/html-stylesheets.html for how to\ncustomize this style sheet.\n*/\n\n/* used to remove borders from tables and images */\n.borderless, table.borderless td, table.borderless th {\n  border: 0 }\n\ntable.borderless td, table.borderless th {\n  /* Override paddin